In [4]:
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/ondrejblaha/Library/Python/3.9/lib/python/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/ondrejblaha/Library/Python/3.9/lib/python/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/Users/ondrejblaha/Library/Python/3.9/lib/python/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/Users/ondrejblaha/Library/Python/3.9/lib/python/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/Libra

In [1]:
#%pip install beautifulsoup4

import requests
import os
import subprocess
from urllib.parse import urljoin
from bs4 import BeautifulSoup

/Users/ondrejblaha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def download_ts_files(webpage_url, output_dir='ts_files'):
    os.makedirs(output_dir, exist_ok=True)
    
    response = requests.get(webpage_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all .ts file links 
    ts_links = [link.get('src') for link in soup.find_all(src=lambda x: x and x.endswith('.ts'))]
    
    downloaded_files = []
    for link in ts_links:
        full_url = urljoin(webpage_url, link)
        file_response = requests.get(full_url)
        filename = os.path.join(output_dir, os.path.basename(link))
        
        with open(filename, 'wb') as f:
            f.write(file_response.content)
        
        downloaded_files.append(filename)
    
    return downloaded_files

def merge_ts_files(input_dir, output_filename='output.mp4'):
    with open('ts_file_list.txt', 'w') as f:
        ts_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.ts')])
        for file in ts_files:
            f.write(f"file '{os.path.join(input_dir, file)}'\n")
    
    subprocess.run([
        'ffmpeg', 
        '-f', 'concat', 
        '-safe', '0', 
        '-i', 'ts_file_list.txt', 
        '-c', 'copy', 
        output_filename
    ])
    
    os.remove('ts_file_list.txt')



In [3]:
# Directly executable in Jupyter
webpage_url = 'https://itvs24.cz/kunovice/softbalovi-juniori-snails-hrali-o-medaile'
ts_files = download_ts_files(webpage_url)

OSError: [Errno 30] Read-only file system: 'ts_files'

In [ ]:
merge_ts_files('ts_files', 'output.mp4')